# The Big Sleep: BigGAN+CLIP

https://openai.com/blog/clip/

https://github.com/lucidrains/big-sleep


![CLIP 1](https://openaiassets.blob.core.windows.net/$web/clip/draft/20210104b/overview-a.svg "CLIP 1")
![CLIP 2](https://openaiassets.blob.core.windows.net/$web/clip/draft/20210104b/overview-b.svg "CLIP 2")

In [ ]:
#@title Imports
!pip install big-sleep --upgrade

from tqdm.notebook import trange
from IPython.display import Image, display

from big_sleep import Imagine
import os
from pathlib import Path

import moviepy.editor as mpy
import glob


In [ ]:
#@title Test Big Sleep
#@markdown Text to visualize
txt = 'green dog on the snow' #@param {type:"string"}

#@markdown Number of epochs
epochs = 10 #@param {type:"integer"}
#@markdown Number of iterations per epoch
iterations = 500 #@param {type:"integer"}
#@markdown Save every
save_every = 50#@param {type:"integer"}

model = Imagine(
    text = txt,
    save_every = save_every,
    lr = 5e-2,
    iterations = iterations,
    save_progress = True,
)
path = "/content/test_images"
Path(path).mkdir(parents=True, exist_ok=True)
os.chdir(path)
for epoch in trange(epochs, desc = 'epochs'):
    for i in trange(iterations, desc = 'iteration'):
        model.train_step(epoch, i)

        if i == 0 or i % model.save_every != 0:
            continue

        filename = txt.replace(' ', '_')
        image = Image(f'{path}/{filename}.png')
        display(image)

In [ ]:
#@title Load text
#@markdown Drag-n-drop txt file to the Files sidebar, paste the path to the file you want to use, then run this cell
text_file = '/content/gpt2_gentext_20210412_070539.txt' #@param {type:"string"}

#@markdown Number of epochs
epochs = 1 #@param {type:"integer"}
#@markdown Number of iterations per epoch
iterations = 50 #@param {type:"integer"}
#@markdown Save every
save_every = 5#@param {type:"integer"}

with open(text_file, 'r') as f:
  txt = f.read()
txt = ". ".join(txt.replace('\n', ' ').split('===================='))
txt_splitted = [t for t in txt.replace('?', '.').replace('!', '.').split('.') if t!='']
Path("/content/text_vis").mkdir(parents=True, exist_ok=True)
for s, sent in enumerate(txt_splitted):
  path = f"/content/text_vis/{s}"
  Path(path).mkdir(parents=True, exist_ok=True)
  os.chdir(path)
  model = Imagine(
      text = sent.replace('/',' '),
      save_every = save_every,
      lr = 5e-2,
      iterations = iterations,
      save_progress = True,
  )
  for epoch in trange(epochs, desc = 'epochs'):
    for i in trange(iterations, desc = 'iteration'):
        model.train_step(epoch, i)
  

In [ ]:
#@title Make clips
#@markdown This cell joins generated images to videoclip for each sentence. Then merge them

#@markdown FPS
fps = 10 #@param {type:"integer"}
Path("/content/vids").mkdir(parents=True, exist_ok=True)
clips = []
for i in trange(len(txt_splitted), desc = 'clips'):
  if Path(f"/content/text_vis/{i}").is_dir():
    images_list = sorted(glob.glob(f"/content/text_vis/{i}/*.png"))
    clip = mpy.ImageSequenceClip(images_list, fps=fps)
    clip.write_videofile(f"/content/vids/{i:03}.mp4")
    clips.append(clip)

final = mpy.concatenate_videoclips(clips)
final.write_videofile("/content/result.mp4")



